In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()

        self.conv_block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False), # kernel_size=3, padding=1
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False), # kernel_size=3, padding=1
            nn.BatchNorm2d(out_channels)
        )

        self.relu = nn.ReLU(inplace=True)

        self.downsample = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(out_channels)
        ) if stride != 1 or in_channels != out_channels else None

    def forward(self, x):
        residual = x
        out = self.conv_block(x)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


In [3]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        #self.in_channels = 64
        self.in_channels = 128
        channels = 128
        self.initial = nn.Sequential(
            nn.Conv2d(3, channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(channels),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        )
        self.layer1 = self.make_layer(block, channels, layers[0])
        self.layer2 = self.make_layer(block, channels * 2, layers[1], stride=2)
        self.layer3 = self.make_layer(block, channels * 4, layers[2], stride=2)
        self.layer4 = self.make_layer(block, channels * 8, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        self.dropout = nn.Dropout(0.5)

        self.fc = nn.Linear(channels * 8, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.initial(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)

        x = self.dropout(x)

        x = self.fc(x)
        return x

In [4]:
def find_max_index(predictions, target):
    predicted_index = predictions.index(max(predictions))
    return predicted_index == target

def train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, epochs, save_path):
    for epoch in range(epochs):
        model.train()
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        with torch.no_grad():
            model.eval()
            test_loss = 0.0
            acc = 0
            total = 0

            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                output_list = output.cpu().detach().numpy().tolist()
                target_list = target.cpu().detach().numpy().tolist()

                for i in range(len(data)):
                    if find_max_index(output_list[i], target_list[i]):
                        acc += 1

                total += len(data)
                test_loss += criterion(output, target).item()

            average_loss = test_loss / len(test_loader)
            accuracy = acc / total

            print(f'Epoch: {epoch+1} | Train loss: {loss.item():.4f} | Test loss: {average_loss:.4f} | Accuracy: {accuracy:.4f}')
            
            # Save model
            if (epoch % 2 == 0):
                state = {
                    'epoch': epoch + 1,
                    'state_dict': model.state_dict()
                    #'optimizer': optimizer.state_dict()
                }
                torch.save(state, os.path.join(save_path, f'adam_model_epoch_{epoch+1}.pth'))


In [5]:
# Hyperparameters
batch = 128
lr = 0.001
epochs = 25

save_path = './saved_models_1'
os.makedirs(save_path, exist_ok=True)

In [6]:
# Data loading and preprocessing
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=1),

    transforms.ToTensor(),
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch, shuffle=True)

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
# Model, loss function, and optimizer
model = ResNet(ResidualBlock, [2, 2, 2, 2]).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

print("Результаты:")
# Training and evaluation
train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, epochs, save_path)

Результаты:


/home/n1sko/.local/lib/python3.10/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch: 1 | Train loss: 1.1094 | Test loss: 1.2067 | Accuracy: 0.5610
Epoch: 2 | Train loss: 1.1181 | Test loss: 0.9689 | Accuracy: 0.6565
Epoch: 3 | Train loss: 1.0245 | Test loss: 0.9072 | Accuracy: 0.6856
Epoch: 4 | Train loss: 0.7531 | Test loss: 0.7877 | Accuracy: 0.7551
Epoch: 5 | Train loss: 0.5004 | Test loss: 0.6460 | Accuracy: 0.7833
Epoch: 6 | Train loss: 0.8144 | Test loss: 0.5692 | Accuracy: 0.8051
Epoch: 7 | Train loss: 0.5269 | Test loss: 0.5113 | Accuracy: 0.8289
Epoch: 8 | Train loss: 0.4429 | Test loss: 0.5426 | Accuracy: 0.8193
Epoch: 9 | Train loss: 0.6490 | Test loss: 0.4454 | Accuracy: 0.8536
Epoch: 10 | Train loss: 0.3432 | Test loss: 0.3971 | Accuracy: 0.8678
Epoch: 11 | Train loss: 0.5563 | Test loss: 0.4161 | Accuracy: 0.8622
Epoch: 12 | Train loss: 0.4189 | Test loss: 0.3878 | Accuracy: 0.8701
Epoch: 13 | Train loss: 0.2659 | Test loss: 0.4183 | Accuracy: 0.8641
Epoch: 14 | Train loss: 0.2704 | Test loss: 0.4125 | Accuracy: 0.8672
Epoch: 15 | Train loss: 0.238